In [17]:
import pandas as pd
from lib import standardize_item_numbers

In [18]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [19]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_22320\153542535.py:2: DtypeWarning: Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [20]:
df = df.drop_duplicates(subset=["offender_race", "offender_gender", "offender_age", "item_number"])

In [21]:
df['occurred_date_time'] = sorted(pd.to_datetime(df['occurred_date_time']))
df['hour'] = df['occurred_date_time'].dt.hour
df = df[~((df.hour.fillna("") == ""))]
df.loc[:, "hour"] = df.hour.astype(int)

In [22]:
df.hour.value_counts()

df = df[df.hour.astype(str).isin(["20", "21", "22", "23", "0", "1", "2", "3", "4", "5"])]
df

,item_number,district,location,disposition,signal_type,signal_description,occurred_date_time,charge_code,charge_description,offender_race,...,victim_fatal_status,hate_crime,report_type,year,offenderid,offenderstatus,persontype,Signal Code,rtcc_requested,hour
2,k1775022,8,002XX Bourbon St,OPEN,21,MISCELLANEOUS INCIDENT,2018-01-26 02:00:00,NaN,NaN,NaN,...,Non-fatal,NaN,Incident Report,2022,NaN,NaN,INTERVIEW,NaN,1,2
20,k1592122,8,Iberville St & Clinton St,OPEN,103,DISTURBANCE,2018-03-28 04:00:00,NaN,NaN,NaN,...,Non-fatal,NaN,Supplemental Report,2022,NaN,NaN,VICTIM,NaN,1,4
24,k1495922,3,063XX Peoples Av,OPEN,30S,HOMICIDE (SHOOTING),2018-03-29 01:47:00,14 30.1,SECOND DEGREE MURDER,UNKNOWN,...,Fatal,NaN,Incident Report,2022,789346.0,NaN,VICTIM,NaN,1,1
28,k1491622,8,002XX Loyola Av,OPEN,43,SIMPLE RAPE,2018-04-01 04:30:00,NaN,NaN,NaN,...,Non-fatal,NaN,Incident Report,2022,NaN,NaN,VICTIM,NaN,1,4
34,k1351722,3,039XX Clematis St,OPEN,55,AGGRAVATED CRIMINAL DAMAGE,2018-04-02 01:00:00,14 55,AGG. CRIMINAL DAMAGE,UNKNOWN,...,Non-fatal,NaN,Incident Report,2022,789201.0,NaN,VICTIM,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26721,d0076618,3,52XX Bancroft Dr,OPEN,67A,AUTO THEFT,2022-11-15 05:30:00,NaN,NaN,NaN,...,Non-fatal,NaN,Supplemental Report,2018,NaN,NaN,NaN,67A,1,5
26723,c3547918,8,3XX Bourbon St,OPEN,67P,PICKPOCKET,2022-11-16 00:05:00,14 67.3,UNAUTHORIZED USE OF ACCESS CARD,BLACK,...,Non-fatal,NaN,Supplemental Report,2018,NaN,NaN,NaN,67P,1,0
26728,c3607418,8,3XX Royal St,OPEN,43,SIMPLE RAPE,2022-11-16 22:13:00,NaN,NaN,NaN,...,Non-fatal,NaN,Incident Report,2018,NaN,NaN,NaN,43,1,22
26729,d0539518,8,4XX Bourbon St,OPEN,67P,PICKPOCKET,2022-11-16 22:19:00,14 67,THEFT,BLACK,...,Non-fatal,NaN,Incident Report,2018,NaN,NaN,NaN,67p,1,22


In [23]:
df.district.value_counts()

8    1118
7     583
1     556
5     431
2     420
6     412
4     401
3     358
Name: district, dtype: int64

In [24]:
# df.to_csv("../data/real_time_crime_center/requests_by_hour_of_the_day.csv")